<a href="https://colab.research.google.com/github/peppefdf/CSL_Gipuzkoa/blob/main/Dash_mouse_LatLon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#%pip install osmnx
#%pip install dash
#%pip install dash_leaflet

import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import json
import geopandas as gpd
import dash_leaflet as dl
from dash import Dash, html, dcc, Output, Input, State, callback
from shapely.geometry import Polygon, mapping

center = [43.27310034205181,-2.019703226103527]

app = Dash()
#
#    html.Button(id="btn_run"),

app.layout = html.Div([
    dl.Map([dl.TileLayer(),
           dl.Polygon(positions=[], id='position-data')],
           id='mapa',
           center=center, zoom=14, style={'height': '50vh'}
           ),
])
#    html.P("Coordinate (click on map):"),
#    html.Div(id='out_text')

#@app.callback(Output('out_text', 'children'),
@app.callback(Output('position-data', 'positions'),
              Input('mapa', 'clickData'),
              prevent_initial_call=True)
def on_click(e):
    if e is not None:
        Lat = e['latlng']['lat']
        Lon = e['latlng']['lng']
        map_center = (Lat, Lon)
        distance = 1000
        walk_times = [5,10,15] # in minutes
        # download the street network
        G = ox.graph_from_point(map_center, dist=distance, network_type='walk', simplify=False)
        # get closes graph nodes to origin and destination
        orig_node = ox.nearest_nodes(G, map_center[1], map_center[0])
        polys = []
        for walk_time in walk_times:
           subgraph    = nx.ego_graph(G, orig_node, radius=walk_time, distance='time')
           node_points = [Point(data["x"], data["y"]) for node, data in subgraph.nodes(data=True)]
           #node_points = [[data["y"],data["x"]] for node, data in subgraph.nodes(data=True)]
           convhull = gpd.GeoSeries(node_points).unary_union.convex_hull
           poly_mapped = mapping(convhull)
           poly_coordinates = poly_mapped['coordinates'][0]
           poly = [ [coords[1], coords[0]] for coords in poly_coordinates]
           polys.append(poly)
        return polys[0]
        #return json.dumps(polys[0])

    else:
       pass

if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>